In [1]:
#%pip install sktime==0.11.3

In [2]:
#%pip uninstall tensorflow

In [3]:
#%pip install tensorflow-estimator=2.1
#%pip install tensorflow-gpu=2.1

In [4]:
import numpy as np
import pandas as pd
import timeit
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import fbeta_score, make_scorer
import tensorflow as tf
from pickle import dump
from sklearn.model_selection import KFold


#Import Models
from sktime.classification.hybrid import HIVECOTEV2
from sktime.classification.hybrid import HIVECOTEV1
from sktime.classification.dictionary_based import BOSSEnsemble
from sktime.classification.interval_based import RandomIntervalSpectralEnsemble
from sktime.classification.interval_based import TimeSeriesForestClassifier
from sktime.classification.shapelet_based import ShapeletTransformClassifier
from sktime._contrib.vector_classifiers._rotation_forest import RotationForest

from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier


from sktime.datasets import load_from_arff_to_dataframe as load_arff
#numpy.version.version

In [5]:
if tf.test.is_built_with_cuda():
    print("The installed version of TensorFlow includes GPU support.")
else:
    print("The installed version of TensorFlow does not include GPU support.")

The installed version of TensorFlow includes GPU support.


In [6]:
tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [7]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# Rocket Classifier code

In [8]:
%pip install tsai

Note: you may need to restart the kernel to use updated packages.


In [9]:
#%pip install sklearn --upgrade

In [10]:
import sklearn
from sklearn.linear_model import RidgeClassifierCV, RidgeCV
from sklearn.metrics import make_scorer
from tsai.imports import *
from tsai.data.external import *
from tsai.models.layers import *

D:\Pessoais\AnacondaFiles\envs\teste_with_gpu\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
#export
class RocketClassifier(sklearn.pipeline.Pipeline):
    """Time series classification using ROCKET features and a linear classifier"""
    
    def __init__(self, num_kernels=10_000, normalize_input=True, random_state=None, 
                 alphas=np.logspace(-3, 3, 7), normalize_features=True, memory=None, verbose=False, scoring=None, class_weight=None, **kwargs):
        """
        RocketClassifier is recommended for up to 10k time series. 
        For a larger dataset, you can use ROCKET (in Pytorch).
        scoring = None --> defaults to accuracy.
        
        Rocket args:            
            num_kernels     : int, number of random convolutional kernels (default 10,000)
            normalize_input : boolean, whether or not to normalise the input time series per instance (default True)
            random_state    : Optional random seed (default None)

        """
        try: 
            import sktime
            from sktime.transformations.panel.rocket import Rocket
        except ImportError:
            print("You need to install sktime to be able to use RocketClassifier")
            
        self.steps = [('rocket', Rocket(num_kernels=num_kernels, normalise=normalize_input, random_state=random_state)),
                      ('ridgeclassifiercv', RidgeClassifierCV(alphas=alphas, normalize=normalize_features, scoring=scoring, 
                                                              class_weight=class_weight, **kwargs))]
        store_attr()
        self._validate_steps()

    def __repr__(self):  
        return f'Pipeline(steps={self.steps.copy()})'

    def save(self, fname='Rocket', path='./models'):
        path = Path(path)
        filename = path/fname
        with open(f'{filename}.pkl', 'wb') as output:
            pickle.dump(self, output, pickle.HIGHEST_PROTOCOL)

In [12]:
#export
def load_rocket(fname='Rocket', path='./models'):
    path = Path(path)
    filename = path/fname
    with open(f'{filename}.pkl', 'rb') as input:
        output = pickle.load(input)
    return output

In [13]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [14]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

In [15]:
def create_dt_metrics():
#     columns = ['BOSSEnsemble','KNeighborsTimeSeriesClassifier','ShapeletTransformClassifier',
#                                     'TimeSeriesForestClassifier','HIVECOTE1','HIVECOTE2','RISE',
#                                     'ROCKET']
    columns = ['BOSSEnsemble']
    df_precision = pd.DataFrame(data= None,
                           columns=columns)
    df_accuracy  = pd.DataFrame(data= None,
                           columns=columns)
    df_f1        = pd.DataFrame(data= None,
                           columns=columns)
    df_recall    = pd.DataFrame(data= None,
                           columns=columns)
    df_std       = pd.DataFrame(data= None,
                           columns=columns)
    return df_precision, df_accuracy, df_f1, df_recall, df_std

In [16]:
def populate_df_metrics(precision, accuracy, f1, recall):
    print(f"Precision in populate_df_metrics: {precision}")
    df_precision = df_precision.append(precision, ignore_index=True)
    print(f"Df precision after append: {df_precision}")
    df_accuracy.append(pd.DataFrame(accuracy))
    df_f1.append(pd.DataFrame(f1))
    df_recall.append(pd.DataFrame(recall))
    #return df_precision, df_accuracy, df_f1, df_recall
  
def save_std(df_accuracy,df_std):
    for row in df_accuracy:
        df_std[row] = np.std(df_accuracy[row])
        print(f"STD COL: {df_std[row]}")
    print(f"STD antes do %save: {df_std}")
    df_std.to_csv("results4/" + 'df_all_std.csv', index=False)
    print(f"Save STD: {df_std}")

In [17]:
def create_models(models):
    #     max_ensemble_size=50, max_ensemble_size=3, max_ensemble_size=100
    #     DICA: Geralmente, o max_win_len_prop é o dobro do min_window
    models.append(('BOSSEnsemble',BOSSEnsemble(max_ensemble_size=50, min_window =9, max_win_len_prop=18)))
    
#     #   n_neighbors=1, n_neighbors=5, n_neighbors=10
#     models.append(('KNeighborsTimeSeriesClassifier',KNeighborsTimeSeriesClassifier(n_neighbors=5)))

#     #     n_estimators=200, n_estimators=6, n_estimators=400     
#     models.append(('ShapeletTransformClassifier',ShapeletTransformClassifier(
#         estimator=RotationForest(n_estimators=200),
#         n_shapelet_samples=500,
#         max_shapelets=20,
#         batch_size=100,
#     )))
    
# #     n_estimators=200, n_estimators=6, n_estimators=400   
#     models.append(('TimeSeriesForestClassifier',TimeSeriesForestClassifier(n_estimators=6)))

# #     n_estimators=3, n_estimators=200, n_estimators=400    
#     models.append(('HIVECOTE1',HIVECOTEV1(
#                 stc_params={
#                     "estimator": RotationForest(n_estimators=200),
#                     "n_shapelet_samples": 100,
#                     "max_shapelets": 10,
#                     "batch_size": 20,
#                 },
#                 tsf_params={"n_estimators": 200},
#                 rise_params={"n_estimators":200},
#                 cboss_params={"n_parameter_samples":1, "max_ensemble_size": 1, "min_window":1, "max_win_len_prop":1},
#                 verbose=10
#             )))
    
#  #     n_estimators=3, n_estimators=200, n_estimators=400    
#     models.append(('HIVECOTE2',HIVECOTEV2(
#         stc_params={
#             "estimator": RotationForest(n_estimators=200),
#             "n_shapelet_samples": 100,
#             "max_shapelets": 10,
#             "batch_size": 20,
#         },
#         drcif_params={"n_estimators": 50, "n_intervals": 2, "att_subsample_size": 2},
#         arsenal_params={"num_kernels": 50, "n_estimators": 50},
#         tde_params={
#             "n_parameter_samples": 10,
#             "max_ensemble_size": 3,
#             "randomly_selected_params": 5,
#             "max_win_len_prop":2,
#             "min_window":1,
#         },
#         verbose=10
#     )))
    
# #     n_estimators=50, n_estimators=3, n_estimators=100 
#     models.append(('RISE',RandomIntervalSpectralEnsemble(n_estimators=200)))

# #     num_kernels=10000, num_kernels=5000, num_kernels=20000 
#     models.append(('ROCKET',RocketClassifier(num_kernels=5000)))
    return models

In [18]:
#save metrics
def save_results(df_to_save, name, model, train_val_test):
    #save the model
    print("Saving model...")
    dump(model, open('results4/'+name+'_'+train_val_test+'_model.pkl', 'wb'))
    print("Saving metrics...")
    print(f"Df accuracy: {df_to_save}")
    df_to_save.to_csv("results4/" + 'df_'+name+'_'+train_val_test+'.csv', index=False)
    print("Accuracy of models:")
    print(df_to_save)

In [19]:
import copy
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
kf = RepeatedKFold(n_splits=5, n_repeats=2, random_state=42)
X_train, y_train = load_arff("AllCandidas_TRAIN_V3.arff")
X_val, y_val = load_arff("AllCandidas_VAL_V3.arff")
print(len(X_train))
print(len(X_val))
#X_train1, X_val, y_train1, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
models =  []
models = create_models(models)
def gpu(X, y, kf, models):
    scoring = ['accuracy','precision_macro', 'recall_macro','f1_macro']
    df_precision, df_accuracy, df_f1, df_recall, df_std = create_dt_metrics()
    with tf.device('/device:GPU:0'):
        i= 0
        df_precision, df_accuracy, df_f1, df_recall, df_std = create_dt_metrics()
        print(f"Quantidade de modelos {len(models)}")
        for name, model in models:
            print(f"Treining {name}...")
            print("In trainning...")
            model.fit(X_train, y_train)
            print("In prediction...")
            df_train = cross_validate(model, X_train, y_train, cv=kf, scoring=scoring,return_train_score=True)
            print("Finished prediction!")
            print(df_train.keys())
            save_results(pd.DataFrame(df_train),name, model,'train2')
            print("***********")
            print("score val")
            print()
            df_validation = cross_validate(model, X_val, y_val, cv=kf, scoring=scoring)
            print(df_validation.keys())
            save_results(pd.DataFrame(df_validation), name, model,'val2')


10892
2042


In [ ]:
if tf.config.list_physical_devices('GPU'):
    print("Using MirroredStrategy")
    strategy = tf.distribute.MirroredStrategy()
else:  # Use the Default Strategy
    print("Using Default Strategy")
    strategy = tf.distribute.get_strategy()
with strategy.scope():
    # Do something interesting
    print(tf.Variable(1.))
    gpu(X_train,y_train,kf,models)

Using MirroredStrategy
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
MirroredVariable:{
  0: <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=1.0>
}
Quantidade de modelos 1
Treining BOSSEnsemble...
In trainning...
In prediction...


### Executar isso só no final

In [ ]:
def test_final_model(X_test, y_test, models):
    scoring = ['accuracy','precision_macro', 'recall_macro','f1_macro']
    for name, model in models:
        df_test = cross_validate(model, X_test, y_test, cv=kf, scoring=scoring)
        print(df_test.keys())
        save_results(pd.DataFrame(df_test),name, model,'test')
        print("Finished test!")

In [ ]:
X_test, y_test = load_arff("AllCandidas_TEST_V3.arff")
print(len(X_test))
#test_final_model(X_test,y_test,models) #Executar isso só no final